In [2]:
import math
import timeit

import numpy as np
import numba

Ported to Python from: https://github.com/Shark-ML/Shark/blob/4.1/include/shark/Algorithms/DirectSearch/Operators/Hypervolume/HypervolumeCalculator2D.h

In [3]:
# Test case adapted from: https://github.com/Shark-ML/Shark/blob/4.1/Test/Algorithms/Hypervolume.cpp

x_ref = np.array([11., 11.], dtype=float)
xs = np.array([[0., 1.],
               [0.2787464911, 0.9603647191],
               [0.3549325314, 0.9348919179],
               [0.4220279525, 0.9065828188],
               [0.4828402120, 0.8757084730],
               [0.7687221637, 0.6395828601],
               [0.8067485614, 0.5908948796],
               [0.8424097574, 0.5388374529],
               [0.8757076053, 0.4828417856],
               [0.9065821569, 0.4220293744],
               [0.5908933491, 0.8067496824],
               [0.5388359009, 0.8424107501],
               [0.6852861036, 0.7282739568],
               [0.7282728148, 0.6852873173],
               [0.6395815841, 0.7687232254],
               [0.9348914021, 0.3549338901],
               [0.9603643728, 0.2787476843],
               [0.9824775804, 0.1863808035],
               [0.1863801385, 0.9824777066],
               [1., 0.]], dtype=float)

vol_value = 120.196858

### First attempt

In [86]:
def create_point_type(xs):
    fields = [('i{}'.format(i), xs.dtype) for i in range(xs.shape[1])]
    return np.dtype(fields)

create_point_type(xs)

dtype([('i0', '<f8'), ('i1', '<f8')])

In [87]:
_point_2d_dtype = np.dtype([('i0', xs.dtype), ('i1', xs.dtype)])

def calculate_2d(xs: np.ndarray, x_ref: np.ndarray) -> float:
    if xs.shape[0] == 0:
        return 0.

    # Copy point set and sort along its first dimension.
    xs = xs.view(dtype=_point_2d_dtype).reshape(-1)
    xs.sort(order='i0', axis=0)

    # Perform the integration.
    volume = (x_ref[0] - xs[0]['i0']) * (x_ref[0] - xs[0]['i1'])
              
    last_valid_idx = 0
    for k in range(1, xs.shape[0]):
        dim1_diff = xs[last_valid_idx]['i1'] - xs[k]['i1']
        # skip dominated points
        # point is dominated <=> dim1_diff <= 0
        if dim1_diff > 0:
            volume += (x_ref[0] - xs[k]['i0']) * dim1_diff
            last_valid_idx = k

    return volume

In [88]:
vol = calculate_2d(xs, x_ref)
assert math.isclose(vol_value, vol, rel_tol=1e-6)

In [89]:
%timeit calculate_2d(xs, x_ref)

127 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Improvement 1

In [4]:
def calculate_2d_pre_njit(xs: np.ndarray, x_ref: np.ndarray) -> float:
    if xs.shape[0] == 0:
        return 0.

    # Copy point set and sort along its first dimension.
    sorted_idx = np.argsort(xs[:, 0])
    xs = xs[sorted_idx]

    # Perform the integration.
    volume = (x_ref[0] - xs[0][0]) * (x_ref[0] - xs[0][1])
              
    last_valid_idx = 0
    for k in range(1, xs.shape[0]):
        dim1_diff = xs[last_valid_idx][1] - xs[k][1]
        # skip dominated points
        # point is dominated <=> dim1_diff <= 0
        if dim1_diff > 0:
            volume += (x_ref[0] - xs[k][0]) * dim1_diff
            last_valid_idx = k

    return volume

In [5]:
vol = calculate_2d_pre_njit(xs, x_ref)
assert math.isclose(vol_value, vol, rel_tol=1e-6)

In [6]:
%timeit calculate_2d_pre_njit(xs, x_ref)

67.7 µs ± 3.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Improvement 2 (using Numba)

In [73]:
@numba.njit
def calculate_2d_njit(xs: np.ndarray, x_ref: np.ndarray) -> float:
    if xs.shape[0] == 0:
        return 0.

    # Copy point set and sort along its first dimension.
    sorted_idx = np.argsort(xs[:, 0])
    xs = xs[sorted_idx]

    # Perform the integration.
    volume = (x_ref[0] - xs[0][0]) * (x_ref[0] - xs[0][1])
              
    last_valid_idx = 0
    for k in range(1, xs.shape[0]):
        dim1_diff = xs[last_valid_idx][1] - xs[k][1]
        # skip dominated points
        # point is dominated <=> dim1_diff <= 0
        if dim1_diff > 0:
            volume += (x_ref[0] - xs[k][0]) * dim1_diff
            last_valid_idx = k

    return volume

In [75]:
vol = calculate_2d_njit(xs, x_ref)
assert math.isclose(vol_value, vol, rel_tol=1e-6)

In [77]:
%timeit calculate_2d_njit(xs, x_ref)

2.43 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Inclusion-Exclusion baseline

Ported from Shark

In [13]:
def hv_inclusion_exclusion(ps: np.ndarray, ref_p: np.ndarray) -> float:
    
    def _helper(ps, ref_p, p, setsize=0, i=0):
        if i == ps.shape[0]:
            if setsize > 0:
                vol = 1.
                for a,b in zip(p, ref_p):
                    vol *= b - a
                return vol if (setsize & 1) else -vol
            else:
                return 0.
        else:
            vol = _helper(ps, ref_p, p, setsize, i + 1)
            p = np.maximum(p, ps[i])
            vol += _helper(ps, ref_p, p, setsize + 1, i + 1)
            return vol
    
    p = np.full_like(ref_p, 1e-6, dtype=float)

    return _helper(ps, ref_p, p)

In [14]:
hv_inclusion_exclusion(xs, x_ref)

120.19683764929286